In [16]:
import boto3
import pandas as pd
from botocore.exceptions import NoCredentialsError, ClientError
import os

In [17]:
# Create Client with environment variables to avoid hard-coding sensitive info
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

s3_client = boto3.client(
            's3',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key
    )


In [18]:
#List objects in source bucket
def list_objects(bucket_name: str, s3_client):
    file_list = []
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)

        if 'Contents' in response:
            for obj in response['Contents']:
                file_list.append(obj['Key'])

    except ClientError as e:
        print(f"ClientError: {str(e)}")

    except Exception as e:
        print(f"an unexcpected {str(e)}")

    return file_list

In [19]:
# Function to download CSV from s3
def download_csv_from_s3(bucket_name: str, file_key: str, local_file_path: str, s3_client):
    try:
        s3_client.download_file(bucket_name, file_key, local_file_path)
        print(f"The file {file_key} has been successfully downloaded to {local_file_path}")
    except Exception as e:
        print(f"Following exception was caught: {str(e)}")
    except ClientError as c:
        print(f"The following error occured: {str(c)}")

In [20]:
# Function to load CSV into DataFrame
def load_csv_to_dataframe(local_file_path: str, file_name: str):
    full_path = local_file_path + '/' + file_name
    df = pd.read_csv(full_path)
    return df

In [21]:
# Set variables to be used for download functionality
bucket_name = 'mindex-data-analytics-code-challenge'
file_keys = list_objects(bucket_name=bucket_name, s3_client=s3_client)
local_file_path = input("Enter destination file path (ex: C:/home/desktop/)")
if not local_file_path.endswith('/'):
    local_file_path = local_file_path + '/'

In [22]:
# Iterate through file list and download csv into local path
for i in file_keys:
    full_path = local_file_path + i
    download_csv_from_s3(bucket_name=bucket_name, file_key=i, local_file_path=full_path, s3_client=s3_client)

The file bengals.csv has been successfully downloaded to C:/Users/Jenja/Desktop/mindex/files/bengals.csv
The file boyd_receiving.csv has been successfully downloaded to C:/Users/Jenja/Desktop/mindex/files/boyd_receiving.csv
The file chase_receiving.csv has been successfully downloaded to C:/Users/Jenja/Desktop/mindex/files/chase_receiving.csv
The file higgins_receiving.csv has been successfully downloaded to C:/Users/Jenja/Desktop/mindex/files/higgins_receiving.csv


In [23]:
# Create DataFrames from CSV files that were downloaded
bengals_df = load_csv_to_dataframe(local_file_path=local_file_path,file_name='bengals.csv')
boyd_rec_df = load_csv_to_dataframe(local_file_path=local_file_path,file_name='boyd_receiving.csv').add_prefix('Boyd_')
chase_rec_df = load_csv_to_dataframe(local_file_path=local_file_path,file_name='chase_receiving.csv').add_prefix('Chase_')
higgins_rec_df = load_csv_to_dataframe(local_file_path=local_file_path,file_name='higgins_receiving.csv').add_prefix('Higgins_')

In [24]:
# Merge df's into one unified dataframe, update Result, select neccessary columns and display
merged_df = chase_rec_df.merge(boyd_rec_df, how="left", left_on="Chase_Week", right_on="Boyd_Week")
merged_df = merged_df.merge(higgins_rec_df, how="left", left_on="Chase_Week", right_on="Higgins_Week")
merged_df = bengals_df.merge(merged_df, how="left", left_on="Week", right_on="Chase_Week")
merged_df.loc[merged_df["Result"] == 1, "Result"] = "W"
merged_df.loc[merged_df["Result"] == 0, "Result"] = "L"
merged_df = merged_df[["Week", "Opponent", "Location", "Result", "Chase_Yards", "Chase_TD", "Boyd_Yards", "Boyd_TD", "Higgins_Yards", "Higgins_TD"]]
display(merged_df)

C:\Users\jenja\AppData\Local\Temp\ipykernel_22220\3914296503.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'W' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged_df.loc[merged_df["Result"] == 1, "Result"] = "W"


,Week,Opponent,Location,Result,Chase_Yards,Chase_TD,Boyd_Yards,Boyd_TD,Higgins_Yards,Higgins_TD
0,PRE1,TB,Away,W,NaN,NaN,NaN,NaN,NaN,NaN
1,PRE2,WSH,Away,L,NaN,NaN,NaN,NaN,NaN,NaN
2,PRE3,MIA,Home,L,NaN,NaN,NaN,NaN,NaN,NaN
3,REG1,MIN,Home,W,101.0,1.0,32.0,0.0,58.0,1.0
4,REG2,CHI,Away,L,54.0,1.0,73.0,0.0,60.0,1.0
5,REG3,PIT,Away,W,65.0,2.0,36.0,1.0,NaN,NaN
6,REG4,JAX,Home,W,77.0,0.0,118.0,0.0,NaN,NaN
7,REG5,GB,Home,L,159.0,1.0,24.0,0.0,32.0,0.0
8,REG6,DET,Away,W,97.0,0.0,7.0,0.0,44.0,0.0
9,REG7,BAL,Away,W,201.0,1.0,39.0,0.0,62.0,0.0


In [25]:
# Query through Pandas to demonstrate Receiver totals
# In SQL Query would be as follows: 
# SELECT 
#   SUM(Chase_Yards), 
#   SUM(Boyd_Yards),
#   SUM(Higgins_Yards), 
#   CONCAT(SUM(CASE WHEN Result = 'W' THEN 1 ELSE 0 END),
#           '-',
#            CASE WHEN Result = 'L' THEN 1 ELSE 0 END)) AS Record
# FROM devon_adjei

chase_sum = merged_df['Chase_Yards'].sum()
boyd_sum = merged_df['Boyd_Yards'].sum()
higgins_sum = merged_df['Higgins_Yards'].sum()
count_wins = (merged_df['Result'] == 'W').sum()
count_losses = (merged_df['Result'] == 'L').sum()
record = f'{count_wins} - {count_losses}'

totals_df = pd.DataFrame({
    'Boyd_Yards': [boyd_sum],
    'Chase_Yards': [chase_sum],
    'Higgins_Yards': [higgins_sum],
    'Record': [record]
})

display(totals_df)


,Boyd_Yards,Chase_Yards,Higgins_Yards,Record
0,938.0,1823.0,1400.0,14 - 10


In [26]:
# Write DataFrame to PostgresSQL DB
from sqlalchemy import create_engine

db_user = 'devon_adjei'
db_password = 'aevonddjei'
db_host = 'ls-2619b6b15c9bdc80a23f6afb7eee54cf0247da21.ca3yee6xneaj.us-east-1.rds.amazonaws.com'
db_port = '5432'
db_name = 'postgres'

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)
merged_df.to_sql('devon_adjei', engine, if_exists='replace', index=False)

print("Data has been inserted into Table.")

ModuleNotFoundError: No module named 'sqlalchemy'